<font size=6em color="red">Student Group Description</font>

Update the table below with your data.

| Matriculation Number | Name | Contribution in this notebook |
| :--- | :--- | :--- |
| 99999 | Alice Wonderland | Example: 50% of all |
| 99998 | John Doe | Example: 50% of Task 3.2  |


# Task02: Compare Irradiance Data of Different Sources

Compare different global radiation data sources for the Green Fab Lab location and your individual three days period in the year 2022.

You will work with 4 data sources:

1. The global radiation (global horizontal irradiance, GHI) measurements at our **[HSRW weather station](https://wiki.eolab.de/doku.php?id=eolab:weather_station:kamp-lintfort:start)**,
2. **[GHI measurements of the DWD](https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/10_minutes/solar/)** for one of three stations nearest to Kamp-Lintfort,
3. **Typical Meteorological Year (TMY)** data from the **[Photovoltaic Geographical Information System (PVGIS)](https://re.jrc.ec.europa.eu/pvg_tools/en/)** of the European Commission.
4. GHI under clear sky conditions generated with  **[pvlib](https://pvlib-python.readthedocs.io/en/v0.11.0/index.html)**.  


## Imports

In [ ]:
import DWD_helpers as dwd
import my_helpers as my
import pvlib

import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# Just a demo on time zones and day saving times (DST). 
# Time zone "CET" = "Europe/Berlin"
# In winter: CET = UTC+0100
# In summer: CET = UTC+0200
print("WINTER TIME")
display(pd.Timestamp("2022-03-01").tz_localize("CET")) # Central European Time
display(pd.Timestamp("2022-03-01").tz_localize("Europe/Berlin")) # Central European Time
print("SUMMER TIME")
display(pd.Timestamp("2022-07-01").tz_localize("CET")) # Central European Summer Time 
display(pd.Timestamp("2022-07-01").tz_localize("Europe/Berlin")) # Central European Summer Time 

## 1. Generate Your Individual Date Range for the Data Analysis 

The execution of `start_date = my.matnos_to_date(matno1, matno2, seed)` further below yields a random timestamp, which is the start date for your analysis. The end date is three days later. The function considers two matriculation numbers and a so-called `seed`, which can be changed to generate another date from the matriculation numbers should it be necessary. 

In case only a single student is working on the problem just set `matno1` to your matriculation number. Set the second to `matno2=99999`. 

Example:<br>
Input:
```
start_date = my.matnos_to_date(matno1=54321, matno2=99999, seed=0)[0]
```
Output:
```
start_date = Timestamp('2022-03-10 00:00:00+0100', tz='CET')
end_date   = Timestamp('2022-03-13 00:00:00+0100', tz='CET')
```

It may be that for your combination of `matno1, matno2, seed=0` leads to a date for which the data at the HSRW weather station is missing. Just change the seed to another integer until you get a date for which HSRW weather station data is available, e.g. `matno1, matno2, seed=5`.

If you found an appropriate date range keep the seed fixed!

We provide a dynamic URL to the Grafana dashboard of the HSRW weather station with a query string that contains your current date range. This helps you to check if you have continous data during the three consecutive days. **If not, change the seed and check again!** 

In [ ]:
help(my.matnos_to_date)

**Fill in your real matriculation numbers.**

In [ ]:
matno1=54321 # Replace with your matriculation number (matno)
matno2=99999 # Replace with the second student's matno. Set it to 99999 if you work alone.

Start with `seed=0`, **Change the seed in case you get an inappropriate date range with missing data.** The `seed` will change the generated analysis date range randomly.

In [ ]:
seed=0

In [ ]:
# The date range generator
start_date = my.matnos_to_date(matno1, matno2, seed)[0]
end_date = start_date + pd.Timedelta(days=3)

print("Your individual date interval for the data analysis:")
print(f"{start_date = }")
print(f"{end_date   = }")

**Check your date range: Is contiguous global radiation data available?**

<img src="./images/HSRW_Station_Data_Gaps_001.png" width= 600 />

_Fig.: Global radiation data at the HSRW weather station with data gaps. In the linear sections data is missing._

The above figure shows the effects of missing data in the graphs.

**Click on the link below** to check the **availaility of global radiation data** and the **absence of data gaps** at the HSRW weather station. If the data is not available or not contiguous (revealing gaps), **change the seed and repeat the procedure** until you find an appropriate time range with valid global radiation data at the HSRW weather station.

In [ ]:
HSRW_grafana_URL = my.gen_HSRW_weather_station_grafana_URL(start_date, end_date)
print("CLICK ON THE LINK TO CHECK YOUR DATA!")
display(HSRW_grafana_URL)

#### Q 1.1 (1P): You successfully found your individual date range for the further data analysis. 

**Answer:** (yes|no)

## 2. Get the GHI from the HSRW weather station

We provide a so-called RESTful API, which allows you to extract data from the weather station database. [**Read here**](https://wiki.eolab.de/doku.php?id=eolab:weather_station:kamp-lintfort:start#restful_api_to_request_real-time_online_data) how to use it to request data.

The following **RESTful API endpoint** (special URL) provides an example:
<br>
**https://weather.eolab.de/api/weather/2021-11-12T14:55:32.000+0100/2021-11-12T14:59:32.000+0100/**

If you click on the link the RESTful API answers to the request with the data 
<br> between `2021-11-12T14:55:32.000+0100` and `2021-11-12T14:59:32.000+0100`.

The first rows look like 
```
[{"wind_speed":"0.56","wind_direction":"188.00","air_temperature":"8.91","air_relhumidity":"73.59","smp10":"108","pqsl":"193.76","soil_moisture":"23.90","soil_tempblue":"10.86","soil_tempred":"7.90","air_pressure":"1011.24","precipitation":"0.0","created_at":"2021-11-12T13:59:27.000Z","created_at_tz":"2021-11-12T13:59:28.658Z"},{"wind_speed":"1.09","wind_direction":"188.00","air_temperature":"8.91","air_relhumidity":"73.58","smp10":"108","pqsl":"193.92","soil_moisture":"23.90","soil_tempblue":"10.86","soil_tempred":"7.90","air_pressure":"1011.24","precipitation":"0.0","created_at":"2021-11-12T13:59:16.000Z","created_at_tz":"2021-11-12T13:59:17.880Z"},{"wind_speed":"1.59","wind_direction":"140.00","air_temperature":"8.91","air_relhumidity":"73.55","smp10":"108","pqsl":"193.76","soil_moisture":"23.90","soil_tempblue":"10.86","soil_tempred":"7.90","air_pressure":"1011.24","precipitation":"0.0","created_at":"2021-11-12T13:59:06.000Z","created_at_tz":"2021-11-12T13:59:07.266Z"}]]
```

This is a **list of dictionaries** (aka JSON format). Each dictionary contains a complete dataset of measurement data of all available sensors at one particular timestamp `"created_at"`.

The first dataset is:
```
{
    "wind_speed":"0.56",
    "wind_direction":"188.00",
    "air_temperature":"8.91",
    "air_relhumidity":"73.59",
    "smp10":"108",
    "pqsl":"193.76",
    "soil_moisture":"23.90",
    "soil_tempblue":"10.86",
    "soil_tempred":"7.90",
    "air_pressure":"1011.24"
    ,"precipitation":"0.0",
    "created_at":"2021-11-12T13:59:27.000Z",
    "created_at_tz":"2021-11-12T13:59:28.658Z"
}

```
**The column `smp10` contains the GHI data in W/m²!**



#### Q 2.1 (2P): Create the **right API endpoint (URL)** for your own analysis date range, **download** the data, read it into a **dataframe** and **plot** it. 

Start with the RESTful API endpoint example above, i.e.
<br>`https://weather.eolab.de/api/weather/2021-11-12T14:55:32.000+0100/2021-11-12T14:59:32.000+0100/`
<br>and replace the dates with your start and end dates. 
You can either write the URL directly down (name it `HSRW_WS_URL`) or you can generate it automatically by string concatenation. 
<br>
Hint: The `pandas.Timestamp` class has a function which prints out the timestamp in ISO8601 format. This can be directly used in the RESTful API endpoint.

In [ ]:
# Your code:

# HSRW_URL = "https://..." + "..." + "/" + "..." + "/"
# display(HSRW_URL)



**Download the data:** Either you use the text output (in JSON format) shown on your **browser** and press `<ctrl>-s` to save the data **or use the `requests` module** to achieve this. Save the data under `./data/download.json`.

**Read the data into a dataframe.** 
<br>The JSON data can be read to a pandas dataframe easily: 

In [ ]:
# Your code:

# HSRW_df = pd...
# HSRW_df.head(3)



Make the column `created_at` the dataframe's index permanently. Plot the GHI series of the dataframe.  

In [ ]:
# You code:

# ...
# HSRW_df[...].plot()



## 3. Get the GHI from the DWD

<img src="./images/DWD_Stations_Global_Radiation.png" width = 800/>

_Fig.: Map of the DWD stations around Kamp-Lintfort set as "active" in measuring "sun related variables" with 10 mins resolution._

The above map was created with data from the station description file **[10_minutes/solar/historical/zehn_min_sd_Beschreibung_Stationen.txt](https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/10_minutes/solar/historical/zehn_min_sd_Beschreibung_Stationen.txt)**. 

The stations on the map were not additionally filtered. It would have made sense to consider only active stations, i.e. which have a `bis_datum` column showing a recent date close to now(). Many of the stations shown do not provide data we are currently interested in, anymore.  

Read **[DESCRIPTION_obsgermany_climate_10min_solar_en.pdf](https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/10_minutes/solar/DESCRIPTION_obsgermany_climate_10min_solar_en.pdf)** to understand the data structure of the actual measurement data.

A **nasty complication** is that different data sources based on different measuring instruments are mixed on the data files. The pdf above states:

| Column  | Physical Quantity  | Missing Value  | Unit  | Type  |
| --- | --- | --- | --- | --- |
| GS_10 | sum of global radiation during the previous 10 minutes | -999 | J/cm² |NUMBER |
| SD_10  | sum of sunshine duration during the previous 10 minutes | -999 | h | NUMBER | S

Measuring "sunshine duration" (What is it? How do you define it?) requires a different instrument than measuring "global radiation". The latter requires a pyranometer. 

Stations are claimed to be active if any(!) measurement is provided. Most stations just measure `SD_10` and do not have any measurung instrument for `GS_10`. Still they are marked as active.

This means we have to open every data file we are interested in and check mannually if `GS_10` measurements are provided or not.

#### Q 3.1 (1P): Identify the three DWD stations closest to Kamp-Lintfort. Download and check the radiation data. Plot it. 

From the map above identify the **three DWD stations closest to Kamp-Lintfort**.  What are their names and station IDs? Complete the markdown table below.

Use the following data collection: 10 minutes resolution, solar/sun data, historical data including year 2022.
Download the 2022 data for the three nearest stations from [**here**](https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/10_minutes/solar/historical/) and **check if global radiation data (column `GS_10`) is available**, i.e. not all data are NaN (-999). (`GS_10`: Globalstrahlung (10 mins resolution), global radiation, global horizontal irradiance, GHI).

Fill out the following table with the three nearest stations. The third column can only be filled after you have checked the actual measurement data in the downloaded files.

| No. | Station ID  | Station Name | Provides GS_10 data in 2022 |
| --- |  --- |  --- |  --- |
| 0 |  99999 |  Dummy |  no | 
| 1 | ... | ... | ... |
| 2 | ... | ... | ... |
| 3 | ... | ... | ... |

Download the measurement data of the three closest stations and store the zips under the folder `./data/`. Unzip the zip archives in individual destination folders.

There is **only one station** in the vicinity of Kamp-Lintfort which provides the GHI data (aka "global radiation", "Globalstrahlung", "GS_10") in 2022. **Read the data into a dataframe and plot!** Hint: Investigate [**DWD_helpers.py**](DWD_helpers.py). There is a function which may help you.

Set the **x-axis limits** to your **personal analysis time range**. The the **ylabel** to "GHI" and add text with the **unit** of the data.  

In [ ]:
# Your code: 

# Solar radiation data read from DWD
# DWD_solrad = dwd...
# fig, ax = plt.subplots()
# DWD_solrad.df[...].plot(ax=ax)
# ax.set_xlim...
# ax.set_ylabel("GHI in ...")



#### Q 3.2 (2P): Convert the irradiance units and plot.

If you just plot the GHI data from DWD and HSRW in one diagram you will notice that the **curves do not match.** You have to **convert the units of the DWD irradiance**. 
<br>
The DWD irradiance is given in $\mathrm{J/cm^2}$ accumulated over 10 minutes. Convert it to $\mathrm{W/m^2}$. 
<br>
Plot the corrected curves in one axis. Now they should be very similar.

In [ ]:
# Your code:

# fig, ax = plt.subplots(figsize=(10,5))
# HSRW_df...
# DWD_solrad...
# ax.set_ylabel("GHI in W/m²")
# ax.legend(["HSRW", "DWD"])



## 4. Clear Sky

#### Q 4.1 (2P): Use PVLIB to determine the clear sky irradiance and plot it. 

Use the GFL coordinates (given in coord. ref. system WGS84 (= EPSG:4326)): `GFL_loc = (lat, lon) = (51.498, 6.549)`
for your analysis date range. Where is the GFL? ;-)

In [ ]:
GFL_loc = (lat, lon) = (51.498, 6.549)
print("Clink on the Google Maps link: ", end="")
print(f"https://www.google.de/maps/place/{lat}+{lon}/") # BTW: This is an API endpoint :-)

Create a **pvlib location object** `site_location` first (see Task 01). Create the **date range** with `times=pd.date_range()` with 10 mins resolution for your personal analysis time range. Create the pandas dataframe `clearsky` by calling the appropriate method of the location object.

In [ ]:
# Your code:

# name = "GFL"
# tz = 
# lat = 
# lon = 
# alt = 

# The Location object is needed to call the function get_clearsky()
# site_location = pvlib.location.Location(lat, ..., ..., ..., name)
# times = pd.date_range(...)
# clearsky = ...



In [ ]:
# Your code:

# fig, ax = plt.subplots(figsize=(10,5))
# ...



## 5. Typical Meteorological Year (TMY) from the PVGIS tools



The clear sky assumption does not take weather conditions into account. The performance estimates of PV plants based on the clear sky assumption are generally suitable for comparing different potential sites, i.e. how does the irradiance differ between Kamp-Lintfort and Almería just because of the different geography? For actual performance predictions it would be better to have real site-specific radiation data, but unfortunately the accurate measurement of the radiation components is difficult and expensive. Furthermore to get a reasonable statistical estimate, long irradiance time series would be required.

#### Background Info: TMY

**What is TMY?**
<br>TMY stands for Typical Meteorological Year. It’s a **standardized dataset** that represents **typical weather conditions** for a **specific location** over a year. TMY data captures variations in meteorological parameters such as solar radiation, temperature, humidity, wind speed, and more.
<br>**How is TMY created?**
<br>TMY data is derived from long-term historical weather records (usually spanning 10 years or more). **For each month** of the year, data is selected **from the year that was considered most “typical” for that month**. For example, **January** might be from **2007**, **February** from **2012**, and so on. TMY datasets provide hourly values for every day of a year. 
<br>(With the help of copilot, modified)

For more information on TMY see the [**European Energy Efficiency Platform (E3P)**](https://e3p.jrc.ec.europa.eu/articles/typical-meteorological-year-tmy)

#### Background Info: Radiation Databases

A good alternative to ground based measurements is the usage of **satellite data** to regionalize the irradiance. The ground level irradiance data cannot be observed directly from satellites but estimated by means of available satellite measurements, radiation modeling as well as ground truth (ground measurement) data for calibration.

One solution (aka product) is provided by the [Satellite Application Facility on Climate Monitoring (**CM-SAF**)](https://www.cmsaf.eu/EN/Home/home_node.html). 

The website states: 
<br>
"CM-SAF develops, generates, archives and distributes high-quality satellite-derived products of the global energy & water cycle and related sustained services in support to understand our climate." 
<br>
And:
<br>
"The new CMSAF solar radiation product SARAH-2.1 (PVGIS-SARAH2) has been added to PVGIS with data from 2005 to 2020." 
<br>([PVGIS 5.2 release notes](https://joint-research-centre.ec.europa.eu/pvgis-photovoltaic-geographical-information-system/pvgis-releases/pvgis-52_en)). 

The **Joint Research Center (JRC)** of the European Commission (EC) provides the **[PVGIS tools](https://joint-research-centre.ec.europa.eu/pvgis-photovoltaic-geographical-information-system_en)** for PV systems design. The ground level SARAH-2 radiation data is accessible through PVGIS. 
<br>

Several radiation databases are combined and used by default for different regions. 
The [PVGIS 5.2 Release Notes](https://joint-research-centre.ec.europa.eu/pvgis-photovoltaic-geographical-information-system/pvgis-releases/pvgis-52_en) states: 
<br>"The new default radiation database is a combination of PVGIS-SARAH2 and PVGIS-SARAH (Europe, Africa, Central Asia), PVGIS-NSRDB (Americas) and PVGIS-ERA5 (Nordic countries above 60 N and the rest of the world)."

<img src="./images/PVGIS_Default_Solar_Radiation_Database_100ppi_1.png" width=600 />

_Fig.: Default Solar Radiation Databases in PVGIS 5.2, European Union, 2022_.
<br>(Source: [PVGIS 5.2 Release Notes](https://joint-research-centre.ec.europa.eu/pvgis-photovoltaic-geographical-information-system/pvgis-releases/pvgis-52_en))


PVGIS uses and provides the radiation data in different ways: 

1. the radiation data is directly used in the [**PVGIS interactive web dashboard**](https://re.jrc.ec.europa.eu/pvg_tools/en/).
1. The data is provoded as a [non-inteactive service API](https://joint-research-centre.ec.europa.eu/pvgis-photovoltaic-geographical-information-system/getting-started-pvgis/api-non-interactive-service_en).

The latter method uses web requests to retrieve the data from the central PVGIS radiation database.
The PVLIB Python library uses this PVGIS Web API. The [pvlib.iotools.pvgis](https://pvlib-python.readthedocs.io/en/stable/_modules/pvlib/iotools/pvgis.html) source code shows how this is done. 

**Download the TMY data from PVGIS**. 
* Go to the [**PVGIS interactive web dashboard**](https://re.jrc.ec.europa.eu/pvg_tools/en/)
* Below the map on the left side **fill in lat/lon** of the GFL and click the `GO!`button
* Now the tabbed pages on the right side are activated. Select the tab `TMY`.
* Select the data from `PVGIS-SARAH2` (2005-2020).
* Download as `CSV`.
* Save the file in the folder `/data/`.
* The filename should be `tmy_51.498_6.549_2005_2020.csv`.
* Open the csv file in a **text editor(!)** and **study the data structure**. Look at all the different text segments. Scroll down to the end of the text file, too.

#### Q 5.1 (1P): Use the right PVLIB function to read the PVGIS TMY data file. ####

Read the TMY data and understand the data structure! Print or display the different elements of `tmy`.

In [ ]:
# Your code:

# tmy = pvlib....
# display(type(tmy))
# print(f"{len(tmy) = }")
# display(tmy[1])
# ...



#### Q 5.2 (1P): Convert all times in the tmy dataframe to year 2022 and plot all GHI data together!

This is a bit tricky. **Copilot of GPT do not find the smartest solution**, at least not with my prompts.

**Instead of AI use** the real cool **NI** concentated **on stackoverflow.com!** (NI: Natural intelligence).

**Search** [**stackoverflow**](https://stackoverflow.com/) for `"Change date of a DateTimeIndex"`. One of the solutions suggests to use the function `map()` together with an unnamed `lambda`function performing a `replace`. This is a super smart pythonic solution, because it vectorizes the conversion. Apply the proposed solution but just replace the year of the date. **Try to understand it!**

In [ ]:
# Your code:

# tmy[0].index = ...



In [ ]:
# You code:

# fig, ax = plt.subplots(figsize=(12,6))
# ...



##### Example Output

When you are smart and invest time in formatting your figure could look something like:

<img src="./images/GHI_final_example.png" width=800 />

**Observations** for this particular example output (Other analysis time ranges might give diofferent insights):

* The GHI measurements of the HSRW weather station and the nearest DWD station with GHI data in 10 minutes resolution match very well.
* The predicted clear sky irrandiance maximum is approx. 10% less than the observations for observation days 1 and 2. Why? Is this physical or a mistake`?
* The measured irradiances exceed the statistically estimated TMY irradiances by far for day 1 and 2.
* The measured irradiances match the statistically estimated TMY irradiances partly well for day 3.


#### Q 5.3 (0P): What are your observations for your data?

**Answer:** ...